<a href="https://colab.research.google.com/github/wilberquito/AMLProject/blob/main/AMLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project: Image Classification
## Advanced Machine Learning


> Wilber E. Bermeo Quito 
>
> Judit Quintana Massana
>
> April 2023

## 

In [9]:
import zipfile
from pathlib import Path

In [10]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [12]:
if IN_COLAB:
    ! pip install torchvision
    from google.colab import drive
    drive.mount('/content/drive')

    data_path = Path('/content/drive/MyDrive/AML/dataset_CIFAR10.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall("data")

    data_path = Path('/content/drive/MyDrive/AML/modular.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall(".")

In [13]:
import modular.datasets as datasets

train_transform, valid_transform = datasets.get_transforms(512)

train_transform, valid_transform

(Compose(
     Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=warn)
     RandomHorizontalFlip(p=0.5)
     RandomVerticalFlip(p=0.5)
     GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
     RandomRotation(degrees=[30.0, 70.0], interpolation=nearest, expand=False, fill=0)
     ToTensor()
     Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
 ),
 Compose(
     Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=warn)
     ToTensor()
     Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
 ))